In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import sys
sys.path.append('..')

from src import utils

from os import path
import joblib
import collections

ModuleNotFoundError: No module named 'src'

In [ ]:
MODEL_DATE = '20-04-08'

DATAFOLDER_PROCESSED = '/Users/navarmn/Work/Projects/2020-Corona/dev/covid19-data-processing/data/processed'
DATAFOLDER_RAW = '/Users/navarmn/Work/Projects/2020-Corona/data/raw'

STATES = ['AC', 'AL', 'AM', 'AP', 'BA', 'CE',
         'DF', 'ES', 'GO', 'MA', 'MT', 'MS',
         'MG', 'PA', 'PB', 'PR', 'PE', 'PI',
         'RJ', 'RN', 'RO', 'RS', 'RR', 'SC',
         'SE', 'SP', 'TO']

MODELS = ['linear-polynomial', 'gpr', 'linear-regression']

MODELS_DETAILS = {
    'linear-polynomial': "Liner Regression + Polynomial Features degree 4",
    'gpr': "Gaussian Process Regressor - Kernel RBF",
    'linear-regression': "Liner Regression - degree 1"
}

MODELS_PATH = path.join('..', 'models')

VERSION = 0.1

In [ ]:
import warnings
warnings.filterwarnings("ignore")
from ipywidgets import interactive


def f(state='TO', days_to_train1=7, days_to_train2=14, smoothing1=0.5, smoothing2=0.5, slope1=0.05, slope2=0.05):
    
    
    df_state = utils.download_state(state=state)
    df_state = df_state[df_state['cases'] != 0]
    dayone = df_state[df_state['cases'] != 0].index[0]
    days = np.array(utils.count_days(dayone=dayone, date_string='%d/%m/%Y'))
    X = days.reshape(-1,1)
    y = utils.get_labels(df_state['cases']).reshape(-1,1)
    X, y = utils.check_inputs(X, y)

    DAYS_TO_TRAIN = 7
    forecast_last_7 = holt_predict(utils.porra(df_state.iloc[-DAYS_TO_TRAIN:]), var_future=14, smoothing_level=0.5, smoothing_slope=0.05)
    y_last_7 = forecast_last_7.values
    X_last_7 = np.arange(X[-1], X[-1]+14, 1)
    X_last_7, y_last_7 = utils.check_inputs(X_last_7, y_last_7)
    ds_last_7 = utils.format_date(df_state.iloc[-DAYS_TO_TRAIN:].index.tolist(), date_string_input='%d/%m/%Y')


    
    DAYS_TO_TRAIN_1 = days_to_train1
    days_to_predict = df_state.shape[0] - DAYS_TO_TRAIN_1
    forecast_first_7 = holt_predict(utils.porra(df_state.iloc[:DAYS_TO_TRAIN_1]), var_future=days_to_predict, smoothing_level=smoothing1, smoothing_slope=slope1)
    y_first_7 = forecast_first_7.values
    X_first_7 = X[DAYS_TO_TRAIN_1:]
    X_first_7, y_first_7 = utils.check_inputs(X_first_7, y_first_7)
    ds_first_7 = utils.format_date(df_state.iloc[DAYS_TO_TRAIN_1:].index.tolist(), date_string_input='%d/%m/%Y')
    
    
    DAYS_TO_TRAIN_2 = days_to_train2
    days_to_predict = df_state.shape[0] - DAYS_TO_TRAIN_2
    forecast_first_14 = holt_predict(utils.porra(df_state.iloc[:DAYS_TO_TRAIN_2]), var_future=days_to_predict, smoothing_level=smoothing2, smoothing_slope=slope2)
    y_first_14 = forecast_first_14.values
    X_first_14 = X[DAYS_TO_TRAIN_2:]
    X_first_14, y_first_14 = utils.check_inputs(X_first_14, y_first_14)
    ds_first_14 = utils.format_date(df_state.iloc[DAYS_TO_TRAIN_2:].index.tolist(), date_string_input='%d/%m/%Y')
    
    
    plt.figure(figsize=(7,5))

    plt.plot(X, y, 'black', linewidth=2, label='Real Data')
    plt.plot(X_first_7, y_first_7, label='Crescimento exponencial usando os primeiros {} dias'.format(DAYS_TO_TRAIN_1))
    plt.plot(X_first_14, y_first_14, label='Crescimento exponencial usando os primeiros {} dias'.format(DAYS_TO_TRAIN_2))
    plt.plot(X_last_7, y_last_7, label='Predição futura')

    plt.legend(fontsize=10)
    plt.ylim(0, y[-1]*5)

    plt.savefig('{}-comparativo.png'.format(state))

    plt.show()

    exp_first_7 = {
        'yhat':y_first_7,
        'ds': ds_first_7,
        'state': state,
        'days': DAYS_TO_TRAIN_1,
        'smoothing': smoothing1,
        'slope': slope1
    }
    
    exp_first_14 = {
        'yhat':y_first_14,
        'ds': ds_first_14,
        'state': state,
        'days': DAYS_TO_TRAIN_2,
        'smoothing': smoothing2,
        'slope': slope2
    }

    

In [ ]:
STATES = ['AC', 'AL', 'AM', 'AP', 'BA', 'CE',
         'DF', 'ES', 'GO', 'MA', 'MT', 'MS',
         'MG', 'PA', 'PB', 'PR', 'PE', 'PI',
         'RJ', 'RN', 'RO', 'RS', 'RR', 'SC',
         'SE', 'SP', 'TO']

interactive_plot = interactive(f, state=STATES, days_to_train=(3, 14), smoothing=(0.2, 5, 0.01), slope=(0.01, 3, 0.01))
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

In [ ]:
%matplotlib inline
from ipywidgets import interactive
import matplotlib.pyplot as plt
import numpy as np

def f(m, b):
    plt.figure(2)
    x = np.linspace(-10, 10, num=1000)
    plt.plot(x, m * x + b)
    plt.ylim(-5, 5)
    plt.show()

interactive_plot = interactive(f, m=(-2.0, 2.0), b=(-3, 3, 0.5))
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot